# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import sys
import sys
sys.path.append('../')

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lerwick,60.1545,-1.1494,10.00,83,100,7.96,GB,1712880872
1,1,port-aux-francais,-49.3500,70.2167,6.25,91,98,3.93,TF,1712880873
2,2,christchurch,-43.5333,172.6333,12.40,93,75,9.26,NZ,1712880584
3,3,bredasdorp,-34.5322,20.0403,12.71,76,5,2.97,ZA,1712880876
4,4,ribeira grande,38.5167,-28.7000,15.99,94,96,1.79,PT,1712880877
...,...,...,...,...,...,...,...,...,...,...
552,552,ejura,7.3856,-1.3562,24.61,85,100,1.42,GH,1712881474
553,553,kindu,-2.9500,25.9500,21.36,95,100,1.28,CD,1712881475
554,554,illizi,26.4833,8.4667,19.89,37,1,6.13,DZ,1712881477
555,555,cabedelo,-6.9811,-34.8339,27.37,80,100,3.86,BR,1712881477


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat',
    size="Humidity", 
    geo=True,
    scale=1,
    color="City", 
    title="City Humidity",
    tiles="OSM",
    frame_width=700, 
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
New_Filtered_cities_Column = city_data_df[(city_data_df['Cloudiness'] == 0)
                               & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 12) 
                               & (city_data_df['Wind Speed'] < 4.5)]

New_Filtered_cities_Column



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,les avirons,-21.2418,55.3394,24.05,73,0,2.06,RE,1712880944
157,157,nova vicosa,-17.8919,-39.3719,26.14,82,0,2.84,BR,1712881055
171,171,migne,46.6274,0.3146,12.06,91,0,2.57,FR,1712881074
192,192,luderitz,-26.6481,15.1594,21.52,47,0,1.85,NaN,1712880507
196,196,jindabyne,-36.4167,148.6333,14.76,59,0,0.45,AU,1712881044
237,237,tubarjal,30.4999,38.2160,17.77,54,0,4.17,SA,1712881163
240,240,narrabri,-30.3167,149.7833,23.62,54,0,0.45,AU,1712881166
265,265,prieska,-29.6641,22.7474,12.54,39,0,4.36,ZA,1712880904
295,295,denia,38.8408,0.1057,14.91,80,0,0.89,ES,1712881226
298,298,hancheng,32.5186,112.3522,15.92,84,0,1.82,CN,1712881229


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = New_Filtered_cities_Column[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
63,les avirons,RE,-21.2418,55.3394,73,
157,nova vicosa,BR,-17.8919,-39.3719,82,
171,migne,FR,46.6274,0.3146,91,
192,luderitz,NaN,-26.6481,15.1594,47,
196,jindabyne,AU,-36.4167,148.6333,59,


In [6]:
# Narrow down cities that fit criteria
New_Filtered_cities_Column = city_data_df[(city_data_df['Cloudiness'] == 0)
                               & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 12) 
                               & (city_data_df['Wind Speed'] < 4.5)]

New_Filtered_cities_Column



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,les avirons,-21.2418,55.3394,24.05,73,0,2.06,RE,1712880944
157,157,nova vicosa,-17.8919,-39.3719,26.14,82,0,2.84,BR,1712881055
171,171,migne,46.6274,0.3146,12.06,91,0,2.57,FR,1712881074
192,192,luderitz,-26.6481,15.1594,21.52,47,0,1.85,NaN,1712880507
196,196,jindabyne,-36.4167,148.6333,14.76,59,0,0.45,AU,1712881044
237,237,tubarjal,30.4999,38.2160,17.77,54,0,4.17,SA,1712881163
240,240,narrabri,-30.3167,149.7833,23.62,54,0,0.45,AU,1712881166
265,265,prieska,-29.6641,22.7474,12.54,39,0,4.36,ZA,1712880904
295,295,denia,38.8408,0.1057,14.91,80,0,0.89,ES,1712881226
298,298,hancheng,32.5186,112.3522,15.92,84,0,1.82,CN,1712881229


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
   #Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
  # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

# Set parameters for hotel search
radius = 10000


#Adding "params"
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit": 30
    }

 
    # Add the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
        hotel_name = name_address_json["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

gaigeturi - nearest hotel: 제주 토비스콘도


,City,Country,Lat,Lng,Humidity,Hotel Name
63,les avirons,RE,-21.2418,55.3394,73,
157,nova vicosa,BR,-17.8919,-39.3719,82,
171,migne,FR,46.6274,0.3146,91,
192,luderitz,NaN,-26.6481,15.1594,47,
196,jindabyne,AU,-36.4167,148.6333,59,
237,tubarjal,SA,30.4999,38.2160,54,
240,narrabri,AU,-30.3167,149.7833,54,
265,prieska,ZA,-29.6641,22.7474,39,
295,denia,ES,38.8408,0.1057,80,
298,hancheng,CN,32.5186,112.3522,84,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:

# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, size=50, hover_cols=['City', 'Country', 'Hotel Name'], title='Nearest Hotels')

# Display the map
map_plot

:Points   [Lng,Lat]   (City,Country,Hotel Name)